# データセットの分割を行う

下記、Amazon Forecastのdocにある、電力消費量データを用いる。
https://docs.aws.amazon.com/forecast/latest/dg/getting-started.html


48時間後までを予測するため、48時間前の情報から予測する必要があります。

# 0.データセットのダウンロード

In [1]:
# !wget -P ../input https://docs.aws.amazon.com/forecast/latest/dg/samples/electricityusagedata.zip

## 1.データセット読み込み

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('../input/electricityusagedata.zip', names=['timestamp','demand','client'])

In [5]:
print(df.shape)
print(df['timestamp'].min())
print(df['timestamp'].max())
df.head()

(3241200, 3)
2014-01-01 01:00:00
2015-01-01 00:00:00


,timestamp,demand,client
0,2014-01-01 01:00:00,2.538071,client_0
1,2014-01-01 01:00:00,23.648649,client_1
2,2014-01-01 01:00:00,0.000000,client_2
3,2014-01-01 01:00:00,144.817073,client_3
4,2014-01-01 01:00:00,75.000000,client_4


## 2.データ分割
48時間後(48points)を予測する。  

all:2014-01-01 01:00:00 〜 2015-01-01 00:00:00  

### train1
train:2014-01-01 01:00:00 〜 2014-06-28 23:00:00  
valid:2014-06-29 00:00:00 〜 2014-06-30 23:00:00(48h)

### train2(for pred)
train:2014-01-01 01:00:00 〜 2014-06-30 23:00:00  
valid:2014-07-01 00:00:00 〜 2014-07-02 23:00:00(48h)


## forecast用
・ヘッダなし
train:2014-01-01 01:00:00 〜 2014-06-30 00:00:00

In [11]:
df[df.timestamp < '2014-07-01 00:00:00'].to_csv('./electricityusage_20140101_20140630.csv', index=False, header=False)

In [ ]:
df['client'] = df['client'].str.replace('client_','').astype(int)

In [ ]:
df.head()

## 2.特徴量生成

### 1)時刻型に変換し、特徴量（時間帯、曜日）を抽出

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['num_of_week'] = df['timestamp'].dt.dayofweek

In [ ]:
df['hour'] = df['timestamp'].dt.hour

### 2)clientごとに時間差の値を特徴量にもつ（36hより前である必要がある）

In [ ]:
df["before_36h"] = df.groupby(['client']).shift(36)['demand'].reset_index()['demand']
df["before_37h"] = df.groupby(['client']).shift(37)['demand'].reset_index()['demand']
df["before_38h"] = df.groupby(['client']).shift(38)['demand'].reset_index()['demand']
df["before_48h"] = df.groupby(['client']).shift(48)['demand'].reset_index()['demand']
df["before_72h"] = df.groupby(['client']).shift(72)['demand'].reset_index()['demand']
df["before_96h"] = df.groupby(['client']).shift(96)['demand'].reset_index()['demand']

### 3)clientごとに期間で集約する

In [ ]:
df = df.sort_values(["client","timestamp"]).reset_index()
df['mean_24h'] = df.groupby(['client']).rolling(24)['demand'].mean().reset_index()['demand']
df['var_24h'] = df.groupby(['client']).rolling(24)['demand'].var().reset_index()['demand']

In [ ]:
#df = df.sort_values(["client","timestamp"]).reset_index()
#df['mean_24h_before_36h'] = df.groupby(['client']).rolling(2)['demand'].mean().reset_index()['demand']

In [ ]:
df["mean_24h_before_36h"] = df.groupby(['client']).shift(36)['mean_24h'].reset_index()['mean_24h']
df["var_24h_before_36h"] = df.groupby(['client']).shift(36)['var_24h'].reset_index()['var_24h']

In [ ]:
df[df.client==21].head()

In [ ]:
df[df.client==21].tail()

In [ ]:
feature_col = [
    'client',
    'num_of_week',
    'hour',
    'before_36h',
    'before_37h',
    'before_38h',
    'before_48h',
    'before_72h',
    'before_96h',
    'mean_24h_before_36h',
    'var_24h_before_36h'
]

## 3.データ分割
all:2014-01-01 01:00:00 〜 2015-01-01 00:00:00  
train:2014-01-01 01:00:00 〜 2014-12-29 00:00:00  
valid:2014-12-29 01:00:00 〜 2014-12-30 12:00:00(36h)  
test:2014-12-30 13:00:00 〜 2015-01-01 00:00:00(36h)  

In [ ]:
df_train = df[df.timestamp <= '2014-12-29 00:00:00']

In [ ]:
df_valid = df[(df.timestamp >= '2014-12-29 01:00:00') & (df.timestamp <= '2014-12-30 12:00:00')]

In [ ]:
df_test = df[df.timestamp >= '2014-12-30 13:00:00']

In [ ]:
df_train[df_train.client==21].shape

In [ ]:
df_valid[df_valid.client==21].shape

In [ ]:
df_test[df_test.client==21].shape

In [ ]:
tr_x = df_train[feature_col]
tr_y = df_train['demand']

In [ ]:
va_x = df_valid[feature_col]
va_y = df_valid['demand']

In [ ]:
test_x = df_test[feature_col]
test_y = df_test[['timestamp','client','demand']]

# XGBoost

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

## 4.モデル学習
誤差はRMSEを用いる

https://docs.aws.amazon.com/ja_jp/forecast/latest/dg/metrics.html

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#fit by best params
regressor = xgb.XGBRegressor(n_estimators=50)

In [ ]:
regressor.fit(tr_x, tr_y, eval_metric="rmse", eval_set=[(va_x, va_y)])

## 5.モデル評価

### testデータでのRMSE

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
test_y

In [ ]:
### RMSEを出力
np.sqrt(mean_squared_error(test_y['demand'], regressor.predict(test_x)))

### 重要度の可視化

In [ ]:
xgb.plot_importance(regressor)

## 6.推論（予測）
https://github.com/dmlc/xgboost/blob/master/demo/guide-python/sklearn_examples.py

In [ ]:
regressor.predict(test_x)

## 7.推論結果の可視化
client_21について  
正解データ  
Amazon Foreast  
xgboost  

In [ ]:
df_xgb = pd.DataFrame(regressor.predict(test_x[test_x.client==21]), columns=['xgb'])

In [ ]:
df_xgb

In [ ]:
df_y = pd.DataFrame(test_y[test_y.client==21]['demand'], columns=['demand']).reset_index(drop=True)

In [ ]:
df_y

### Foreastの結果

In [ ]:
#df_forecast = pd.read_csv('../amazon_forecast_official_dev_guide/export_forecast_drop1week/my_forecast_export_drop1week_2021-03-03T07-51-36Z_part1.csv')
df_forecast = pd.read_csv('../input/my_forecast_export_drop1week_2021-03-03T07-51-36Z_part1.csv')




In [ ]:
df_forecast['item_id'].unique()

In [ ]:
df_result = df_forecast[df_forecast.item_id=='client_21'].reset_index(drop=True)

In [ ]:
df_result

In [ ]:
pd.concat([df_result, df_xgb, df_y], axis=1).plot(x='date',figsize=(20,5), grid=True)

# XGBoostの利点
特徴量の重要度がわかる

【デメリット】
コードのデバッグが大変
特徴量を作り込む必要あり
ハイパーパラメータのチューニング
